Given that I incorrectly filtered out activity_types =! or better incorrectly merged on timestamp left and right, I need to replicate the whole process. Logically, there are only 151 rows of labeled data, which is probably not nearly enough. 

In [33]:
import pandas as pd 
import numpy as np

In [108]:
glasses = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/glasses.csv')
smartwatch_sources = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/source_smartwatch_full.csv')
activity_type_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/report_full_activity_type.csv')
smartphone_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/smartphone_full.csv')

In [112]:
# Convert timestamp columns to datetime objects
activity_type_full['timestamp'] = pd.to_datetime(activity_type_full['timestamp'])
smartwatch_sources['timestamp'] = pd.to_datetime(smartwatch_sources['timestamp'])

In [36]:
# Group activity_type_full by timestamp and transform it to align with smartwatch_sources
smartwatch_sources['activity_type'] = activity_type_full.groupby('timestamp')['activity_type'].transform(lambda x: ','.join(x))

# Print or further process the updated smartwatch_sources DataFrame
print(smartwatch_sources)
# Fill NaN values with 0s
smartwatch_sources.fillna(0, inplace=True)

        index         source               timestamp  \
0      164350  accelerometer 2017-06-29 07:46:01.263   
1      164359  accelerometer 2017-06-29 07:46:06.270   
2      164371  accelerometer 2017-06-29 07:46:11.280   
3      164383  accelerometer 2017-06-29 07:46:16.282   
4      164395  accelerometer 2017-06-29 07:46:21.289   
...       ...            ...                     ...   
27405  150404      gyroscope 2017-07-13 20:03:58.587   
27406  163987      gyroscope 2017-07-13 20:05:00.880   
27407  150410      gyroscope 2017-07-13 20:05:00.880   
27408  164176      gyroscope 2017-07-13 20:21:31.322   
27409  150599      gyroscope 2017-07-13 20:21:31.322   

                                             values_1 activity_type  
0            ['-0.3440857', '0.68330383', '9.967361']   Video games  
1             ['-0.4410553', '0.7886505', '9.872787']   In computer  
2              ['0.2496643', '1.4255066', '9.832092']       At home  
3             ['1.0840454', '0.52648926', '9.96

In [28]:
# Filter the DataFrame to keep rows where both "activity" and "duration" are not equal to 0
filtered_data = smartwatch_sources[(smartwatch_sources['activity_type'] != 0)]

# Print or further process the filtered data
print(filtered_data)

      index         source               timestamp  \
0    164350  accelerometer 2017-06-29 07:46:01.263   
1    164359  accelerometer 2017-06-29 07:46:06.270   
2    164371  accelerometer 2017-06-29 07:46:11.280   
3    164383  accelerometer 2017-06-29 07:46:16.282   
4    164395  accelerometer 2017-06-29 07:46:21.289   
..      ...            ...                     ...   
146  164467        gravity 2017-06-29 07:46:51.668   
147  164480        gravity 2017-06-29 07:46:56.713   
148  164494        gravity 2017-06-29 07:47:01.742   
149  164506        gravity 2017-06-29 07:47:06.786   
150  164516        gravity 2017-06-29 07:47:11.812   

                                      values_1 activity_type  
0     ['-0.3440857', '0.68330383', '9.967361']   Video games  
1      ['-0.4410553', '0.7886505', '9.872787']   In computer  
2       ['0.2496643', '1.4255066', '9.832092']       At home  
3      ['1.0840454', '0.52648926', '9.967361']   In computer  
4    ['-1.2897949', '-0.83462524', '

In [37]:
filtered_data

,index,source,timestamp,values_1,activity_type,source_encoded
0,164350,accelerometer,2017-06-29 07:46:01.263,"[-0.3440857, 0.68330383, 9.967361]",Video games,0
1,164359,accelerometer,2017-06-29 07:46:06.270,"[-0.4410553, 0.7886505, 9.872787]",In computer,0
2,164371,accelerometer,2017-06-29 07:46:11.280,"[0.2496643, 1.4255066, 9.832092]",At home,0
3,164383,accelerometer,2017-06-29 07:46:16.282,"[1.0840454, 0.52648926, 9.967361]",In computer,0
4,164395,accelerometer,2017-06-29 07:46:21.289,"[-1.2897949, -0.83462524, 9.338882]",Eat,0
...,...,...,...,...,...,...
146,164467,gravity,2017-06-29 07:46:51.668,"[-3.9818604, -1.0180248, 8.903864]",Work,1
147,164480,gravity,2017-06-29 07:46:56.713,"[-4.002737, -1.0194948, 8.89433]",Walk,1
148,164494,gravity,2017-06-29 07:47:01.742,"[-3.9689453, -1.0320817, 8.908011]",Train,1
149,164506,gravity,2017-06-29 07:47:06.786,"[-0.7809928, -0.5561664, 9.759667]",Walk,1


In [38]:
# Parse values in 'values_1' column to extract numerical values
filtered_data['values_1'] = filtered_data['values_1'].apply(lambda x: [float(val.strip("[]'")) for val in x.split(", ")]) 

AttributeError: 'list' object has no attribute 'split'

In [ ]:
filtered_data

,index,source,timestamp,values_1,activity_type
0,164350,accelerometer,2017-06-29 07:46:01.263,"[-0.3440857, 0.68330383, 9.967361]",Video games
1,164359,accelerometer,2017-06-29 07:46:06.270,"[-0.4410553, 0.7886505, 9.872787]",In computer
2,164371,accelerometer,2017-06-29 07:46:11.280,"[0.2496643, 1.4255066, 9.832092]",At home
3,164383,accelerometer,2017-06-29 07:46:16.282,"[1.0840454, 0.52648926, 9.967361]",In computer
4,164395,accelerometer,2017-06-29 07:46:21.289,"[-1.2897949, -0.83462524, 9.338882]",Eat
...,...,...,...,...,...
146,164467,gravity,2017-06-29 07:46:51.668,"[-3.9818604, -1.0180248, 8.903864]",Work
147,164480,gravity,2017-06-29 07:46:56.713,"[-4.002737, -1.0194948, 8.89433]",Walk
148,164494,gravity,2017-06-29 07:47:01.742,"[-3.9689453, -1.0320817, 8.908011]",Train
149,164506,gravity,2017-06-29 07:47:06.786,"[-0.7809928, -0.5561664, 9.759667]",Walk


In [ ]:
# Convert the 'values_1' column to numeric if possible
filtered_data['values_1'] = filtered_data['values_1'].apply(pd.to_numeric, errors='coerce')

# Pivot the data to create columns for each unique source_encoded
pivot_data = pd.pivot_table(filtered_data, 
                            values='values_1', 
                            index='timestamp', 
                            columns='source')

# Print the pivoted data
print(pivot_data)

source                                         accelerometer  \
timestamp                                                      
2017-06-29 07:46:01.263   [-0.3440857, 0.68330383, 9.967361]   
2017-06-29 07:46:01.324                                  NaN   
2017-06-29 07:46:06.270    [-0.4410553, 0.7886505, 9.872787]   
2017-06-29 07:46:06.354                                  NaN   
2017-06-29 07:46:11.280     [0.2496643, 1.4255066, 9.832092]   
...                                                      ...   
2017-06-29 09:02:41.725  [-0.7789917, -8.111679, -4.8059387]   
2017-06-29 09:02:47.266   [-1.2313843, -8.289093, -4.984192]   
2017-06-29 09:02:53.414   [-1.7970123, -7.545929, -6.339905]   
2017-06-29 09:02:58.493  [-1.1230469, -8.696945, -4.9900513]   
2017-06-29 09:03:03.495    [-1.250061, -8.08905, -5.3499146]   

source                                              gravity  
timestamp                                                    
2017-06-29 07:46:01.263                    

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_52569/4087824921.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['values_1'] = filtered_data['values_1'].apply(pd.to_numeric, errors='coerce')


In [39]:
pivot_data 

source,accelerometer,gravity
timestamp,,
2017-06-29 07:46:01.263,"[-0.3440857, 0.68330383, 9.967361]",NaN
2017-06-29 07:46:01.324,NaN,"[-0.34131828, 0.6573288, 9.778641]"
2017-06-29 07:46:06.270,"[-0.4410553, 0.7886505, 9.872787]",NaN
2017-06-29 07:46:06.354,NaN,"[-0.49383318, 0.6960046, 9.769447]"
2017-06-29 07:46:11.280,"[0.2496643, 1.4255066, 9.832092]",NaN
...,...,...
2017-06-29 09:02:41.725,"[-0.7789917, -8.111679, -4.8059387]",NaN
2017-06-29 09:02:47.266,"[-1.2313843, -8.289093, -4.984192]",NaN
2017-06-29 09:02:53.414,"[-1.7970123, -7.545929, -6.339905]",NaN


In [40]:
# Expand accelerometer column into x, y, z components
accelerometer_components = pivot_data['accelerometer'].apply(pd.Series)
accelerometer_components.columns = ['accel_x', 'accel_y', 'accel_z']

# Expand gravity column into x, y, z components
gravity_components = pivot_data['gravity'].apply(pd.Series)
gravity_components.columns = ['gravity_x', 'gravity_y', 'gravity_z']

# Concatenate the new components with the original DataFrame
pivot_data = pd.concat([pivot_data, accelerometer_components, gravity_components], axis=1)

# Drop the original columns
pivot_data.drop(columns=['accelerometer', 'gravity'], inplace=True)

# Print the updated DataFrame
print(pivot_data)

                          accel_x   accel_y   accel_z  gravity_x  gravity_y  \
timestamp                                                                     
2017-06-29 07:46:01.263 -0.344086  0.683304  9.967361        NaN        NaN   
2017-06-29 07:46:01.324       NaN       NaN       NaN  -0.341318   0.657329   
2017-06-29 07:46:06.270 -0.441055  0.788651  9.872787        NaN        NaN   
2017-06-29 07:46:06.354       NaN       NaN       NaN  -0.493833   0.696005   
2017-06-29 07:46:11.280  0.249664  1.425507  9.832092        NaN        NaN   
...                           ...       ...       ...        ...        ...   
2017-06-29 09:02:41.725 -0.778992 -8.111679 -4.805939        NaN        NaN   
2017-06-29 09:02:47.266 -1.231384 -8.289093 -4.984192        NaN        NaN   
2017-06-29 09:02:53.414 -1.797012 -7.545929 -6.339905        NaN        NaN   
2017-06-29 09:02:58.493 -1.123047 -8.696945 -4.990051        NaN        NaN   
2017-06-29 09:03:03.495 -1.250061 -8.089050 -5.34991

In [41]:
pivot_data

,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z
timestamp,,,,,,
2017-06-29 07:46:01.263,-0.344086,0.683304,9.967361,NaN,NaN,NaN
2017-06-29 07:46:01.324,NaN,NaN,NaN,-0.341318,0.657329,9.778641
2017-06-29 07:46:06.270,-0.441055,0.788651,9.872787,NaN,NaN,NaN
2017-06-29 07:46:06.354,NaN,NaN,NaN,-0.493833,0.696005,9.769447
2017-06-29 07:46:11.280,0.249664,1.425507,9.832092,NaN,NaN,NaN
...,...,...,...,...,...,...
2017-06-29 09:02:41.725,-0.778992,-8.111679,-4.805939,NaN,NaN,NaN
2017-06-29 09:02:47.266,-1.231384,-8.289093,-4.984192,NaN,NaN,NaN
2017-06-29 09:02:53.414,-1.797012,-7.545929,-6.339905,NaN,NaN,NaN


In [42]:
# Remove duplicate columns, if any
pivot_data = pivot_data.loc[:, ~pivot_data.columns.duplicated()]
# Replace NaN values with 0s
pivot_data.fillna(0, inplace=True)

In [43]:
# Group 'filtered_filtered_data' by both 'timestamp' and 'activity_encoded' and aggregate 'duration'
activity_grouped = filtered_data.groupby(['timestamp'])['activity_type'].first()

# Merge 'duration_grouped' with 'pivot_data' based on timestamp index
pivot_data = pivot_data.merge(activity_grouped, how='left', on='timestamp')

# Print the updated pivoted data with 'duration' as a column specific to each activity
print(pivot_data)

                          accel_x   accel_y   accel_z  gravity_x  gravity_y  \
timestamp                                                                     
2017-06-29 07:46:01.263 -0.344086  0.683304  9.967361   0.000000   0.000000   
2017-06-29 07:46:01.324  0.000000  0.000000  0.000000  -0.341318   0.657329   
2017-06-29 07:46:06.270 -0.441055  0.788651  9.872787   0.000000   0.000000   
2017-06-29 07:46:06.354  0.000000  0.000000  0.000000  -0.493833   0.696005   
2017-06-29 07:46:11.280  0.249664  1.425507  9.832092   0.000000   0.000000   
...                           ...       ...       ...        ...        ...   
2017-06-29 09:02:41.725 -0.778992 -8.111679 -4.805939   0.000000   0.000000   
2017-06-29 09:02:47.266 -1.231384 -8.289093 -4.984192   0.000000   0.000000   
2017-06-29 09:02:53.414 -1.797012 -7.545929 -6.339905   0.000000   0.000000   
2017-06-29 09:02:58.493 -1.123047 -8.696945 -4.990051   0.000000   0.000000   
2017-06-29 09:03:03.495 -1.250061 -8.089050 -5.34991

In [44]:
pivot_data

,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z,activity_type
timestamp,,,,,,,
2017-06-29 07:46:01.263,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,Video games
2017-06-29 07:46:01.324,0.000000,0.000000,0.000000,-0.341318,0.657329,9.778641,"At home,Eat"
2017-06-29 07:46:06.270,-0.441055,0.788651,9.872787,0.000000,0.000000,0.000000,In computer
2017-06-29 07:46:06.354,0.000000,0.000000,0.000000,-0.493833,0.696005,9.769447,Walk
2017-06-29 07:46:11.280,0.249664,1.425507,9.832092,0.000000,0.000000,0.000000,At home
...,...,...,...,...,...,...,...
2017-06-29 09:02:41.725,-0.778992,-8.111679,-4.805939,0.000000,0.000000,0.000000,Train
2017-06-29 09:02:47.266,-1.231384,-8.289093,-4.984192,0.000000,0.000000,0.000000,Meeting
2017-06-29 09:02:53.414,-1.797012,-7.545929,-6.339905,0.000000,0.000000,0.000000,Walk


In [45]:
pivot_data.reset_index()

,timestamp,accel_x,accel_y,accel_z,gravity_x,gravity_y,gravity_z,activity_type
0,2017-06-29 07:46:01.263,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,Video games
1,2017-06-29 07:46:01.324,0.000000,0.000000,0.000000,-0.341318,0.657329,9.778641,"At home,Eat"
2,2017-06-29 07:46:06.270,-0.441055,0.788651,9.872787,0.000000,0.000000,0.000000,In computer
3,2017-06-29 07:46:06.354,0.000000,0.000000,0.000000,-0.493833,0.696005,9.769447,Walk
4,2017-06-29 07:46:11.280,0.249664,1.425507,9.832092,0.000000,0.000000,0.000000,At home
...,...,...,...,...,...,...,...,...
146,2017-06-29 09:02:41.725,-0.778992,-8.111679,-4.805939,0.000000,0.000000,0.000000,Train
147,2017-06-29 09:02:47.266,-1.231384,-8.289093,-4.984192,0.000000,0.000000,0.000000,Meeting
148,2017-06-29 09:02:53.414,-1.797012,-7.545929,-6.339905,0.000000,0.000000,0.000000,Walk
149,2017-06-29 09:02:58.493,-1.123047,-8.696945,-4.990051,0.000000,0.000000,0.000000,In bus


In [46]:
pivot_data_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data.csv'
pivot_data.to_csv(pivot_data_pathway, index=False)

In [47]:
import pandas as pd

# Read the CSV file into a DataFrame
smartphone_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/smartphone_full.csv')

# Explode the lists in the 'values' column into separate rows
smartphone_full_expanded = smartphone_full.explode('values')

# Pivot the DataFrame
pivot_table_s = pd.pivot_table(smartphone_full_expanded,
                               index='timestamp',
                               columns='source',
                               values='values',
                               aggfunc='first')

# Rename the columns
pivot_table_s.rename(columns={'audio': 'audio_values', 'step_detector': 'step_detector_values'}, inplace=True)

# Reset index
pivot_table_s.reset_index(inplace=True)

# Define the pathway for the CSV file
pivot_table_s_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s.csv'

# Save the pivoted DataFrame to a CSV file
pivot_table_s.to_csv(pivot_table_s_pathway, index=False)

# Print the file pathway where the CSV file is saved
print("CSV file saved to:", pivot_table_s_pathway)

CSV file saved to: /Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s.csv


In [48]:
# Convert the 'audio_values' column from list to string and remove square brackets
pivot_table_s['audio_values'] = pivot_table_s['audio_values'].astype(str).str.strip('[]')

# Split the 'audio_values' column into separate columns
audio_values_split = pivot_table_s['audio_values'].str.split(',', expand=True)

# Rename the new columns
audio_values_split.columns = ['audio_value_1', 'audio_value_2', 'audio_value_3', 'audio_value_4']

# Concatenate the new columns with the original DataFrame
pivot_table_s_split = pd.concat([pivot_table_s.drop('audio_values', axis=1), audio_values_split], axis=1)

# Print the updated DataFrame
print(pivot_table_s_split)

                      timestamp step_detector_values audio_value_1  \
0       2017-06-29 07:45:16.506              ['1.0']           nan   
1       2017-06-29 07:45:17.407                  NaN         '0.0'   
2       2017-06-29 07:45:18.407                  NaN       '227.0'   
3       2017-06-29 07:45:19.407                  NaN       '590.0'   
4       2017-06-29 07:45:20.446                  NaN      '1724.0'   
...                         ...                  ...           ...   
617948  2017-07-13 20:13:33.671                  NaN      '1956.0'   
617949  2017-07-13 20:13:34.663                  NaN      '2673.0'   
617950  2017-07-13 20:13:35.663                  NaN      '2340.0'   
617951  2017-07-13 20:13:36.668                  NaN      '2046.0'   
617952  2017-07-13 20:13:37.668                  NaN      '2441.0'   

       audio_value_2 audio_value_3 audio_value_4  
0               None          None          None  
1              '0.0'         '0.0'     '32767.0'  
2     

In [49]:
pivot_table_s_split.fillna(0, inplace=True)

In [50]:
# Remove square brackets and single quotes from the values in 'step_detector_values' column
pivot_table_s_split['step_detector_values'] = pivot_table_s_split['step_detector_values'].str.strip("[]").str.replace("'", "")

# Convert the values in 'step_detector_values' column to float
pivot_table_s_split['step_detector_values'] = pivot_table_s_split['step_detector_values'].astype(float)

In [51]:
pivot_table_s_split['audio_value_1'] = pivot_table_s_split['audio_value_1'].str.replace("'", '').astype(float)
pivot_table_s_split['audio_value_2'] = pivot_table_s_split['audio_value_2'].str.replace("'", '').astype(float)
pivot_table_s_split['audio_value_3'] = pivot_table_s_split['audio_value_3'].str.replace("'", '').astype(float)
pivot_table_s_split['audio_value_4'] = pivot_table_s_split['audio_value_4'].str.replace("'", '').astype(float)

In [52]:
# Convert the 'timestamp' column to datetime data type
pivot_table_s_split['timestamp'] = pd.to_datetime(pivot_table_s_split['timestamp'])

In [53]:
pivot_table_s_split.fillna(0, inplace=True)

In [55]:
# Group activity_type_full by timestamp and transform it to align with smartwatch_sources
pivot_table_s_split['activity_type'] = activity_type_full.groupby('timestamp')['activity_type'].transform(lambda x: ','.join(x))

# Print or further process the updated smartwatch_sources DataFrame
print(pivot_table_s_split)
# Fill NaN values with 0s
pivot_table_s_split.fillna(0, inplace=True)

                     timestamp  step_detector_values  audio_value_1  \
0      2017-06-29 07:45:16.506                   1.0            0.0   
1      2017-06-29 07:45:17.407                   0.0            0.0   
2      2017-06-29 07:45:18.407                   0.0          227.0   
3      2017-06-29 07:45:19.407                   0.0          590.0   
4      2017-06-29 07:45:20.446                   0.0         1724.0   
...                        ...                   ...            ...   
617948 2017-07-13 20:13:33.671                   0.0         1956.0   
617949 2017-07-13 20:13:34.663                   0.0         2673.0   
617950 2017-07-13 20:13:35.663                   0.0         2340.0   
617951 2017-07-13 20:13:36.668                   0.0         2046.0   
617952 2017-07-13 20:13:37.668                   0.0         2441.0   

        audio_value_2  audio_value_3  audio_value_4 activity_type  
0             0.00000            0.0            0.0   Video games  
1          

In [56]:
pivot_table_s_split

,timestamp,step_detector_values,audio_value_1,audio_value_2,audio_value_3,audio_value_4,activity_type
0,2017-06-29 07:45:16.506,1.0,0.0,0.00000,0.0,0.0,Video games
1,2017-06-29 07:45:17.407,0.0,0.0,0.00000,0.0,32767.0,In computer
2,2017-06-29 07:45:18.407,0.0,227.0,113.50000,227.0,227.0,At home
3,2017-06-29 07:45:19.407,0.0,590.0,272.33334,590.0,227.0,In computer
4,2017-06-29 07:45:20.446,0.0,1724.0,635.25000,1724.0,227.0,Eat
...,...,...,...,...,...,...,...
617948,2017-07-13 20:13:33.671,0.0,1956.0,5246.24850,32767.0,30.0,0
617949,2017-07-13 20:13:34.663,0.0,2673.0,5246.18160,32767.0,30.0,0
617950,2017-07-13 20:13:35.663,0.0,2340.0,5246.10640,32767.0,30.0,0
617951,2017-07-13 20:13:36.668,0.0,2046.0,5246.02400,32767.0,30.0,0


In [57]:
# Filter the DataFrame to keep rows where both "activity" and "duration" are not equal to 0
filtered_data_smartphone = pivot_table_s_split[(pivot_table_s_split['activity_type'] != 0)]

In [60]:
merged_data_s = filtered_data_smartphone

In [70]:
merged_data_s 

,timestamp,step_detector_values,audio_value_1,audio_value_2,audio_value_3,audio_value_4,activity_type
0,2017-06-29 07:45:16.506,1.0,0.0,0.00000,0.0,0.0,Video games
1,2017-06-29 07:45:17.407,0.0,0.0,0.00000,0.0,32767.0,In computer
2,2017-06-29 07:45:18.407,0.0,227.0,113.50000,227.0,227.0,At home
3,2017-06-29 07:45:19.407,0.0,590.0,272.33334,590.0,227.0,In computer
4,2017-06-29 07:45:20.446,0.0,1724.0,635.25000,1724.0,227.0,Eat
...,...,...,...,...,...,...,...
146,2017-06-29 07:47:36.448,0.0,23886.0,4708.92140,32767.0,45.0,Work
147,2017-06-29 07:47:37.448,0.0,7923.0,4731.71630,32767.0,45.0,Walk
148,2017-06-29 07:47:38.448,0.0,7005.0,4747.72560,32767.0,45.0,Train
149,2017-06-29 07:47:39.449,0.0,1085.0,4722.11200,32767.0,45.0,Walk


In [61]:
# Define the pathway for the CSV file
merged_data_s_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/merged_data_s.csv'

# Save the pivoted DataFrame to a CSV file
merged_data_s.to_csv(merged_data_s_pathway, index=False)

In [63]:
pivot_data_2 = pivot_data.drop(columns = ['activity_type'])

In [122]:
pivot_data_2= pivot_data_2.reset_index()

In [109]:
glasses = glasses.drop(columns= ['NUM', 'EOG_L', 'EOG_R', 'EOG_H', 'EOG_V'])

In [110]:
glasses 

,timestamp,ACC_X,ACC_Y,ACC_Z,GYRO_X,GYRO_Y,GYRO_Z
0,2017/06/29 08:00:00.406,-3242,10396,-6489,9117,2018,-819
1,2017/06/29 08:00:00.426,-4014,11348,-15131,3359,-882,-2009
2,2017/06/29 08:00:00.446,-3223,10918,-8496,-460,211,-2631
3,2017/06/29 08:00:00.466,-5255,10690,-10854,1152,333,-3799
4,2017/06/29 08:00:00.486,-560,14164,-8619,-327,488,-2833
...,...,...,...,...,...,...,...
3975261,2017/07/13 22:20:30.894,-5345,9790,-12500,336,638,-365
3975262,2017/07/13 22:20:30.914,-5280,9682,-12471,581,812,-271
3975263,2017/07/13 22:20:30.934,-5170,9579,-12492,828,963,-214
3975264,2017/07/13 22:20:30.944,-5082,9531,-12473,881,1062,-189


In [111]:
glasses['timestamp'] = pd.to_datetime(glasses['timestamp'])

In [113]:
# Group activity_type_full by timestamp and transform it to align with smartwatch_sources
glasses['activity_type'] = activity_type_full.groupby('timestamp')['activity_type'].transform(lambda x: ','.join(x))

# Print or further process the updated smartwatch_sources DataFrame
print(glasses)
# Fill NaN values with 0s
glasses.fillna(0, inplace=True)

                      timestamp  ACC_X  ACC_Y  ACC_Z  GYRO_X  GYRO_Y  GYRO_Z  \
0       2017-06-29 08:00:00.406  -3242  10396  -6489    9117    2018    -819   
1       2017-06-29 08:00:00.426  -4014  11348 -15131    3359    -882   -2009   
2       2017-06-29 08:00:00.446  -3223  10918  -8496    -460     211   -2631   
3       2017-06-29 08:00:00.466  -5255  10690 -10854    1152     333   -3799   
4       2017-06-29 08:00:00.486   -560  14164  -8619    -327     488   -2833   
...                         ...    ...    ...    ...     ...     ...     ...   
3975261 2017-07-13 22:20:30.894  -5345   9790 -12500     336     638    -365   
3975262 2017-07-13 22:20:30.914  -5280   9682 -12471     581     812    -271   
3975263 2017-07-13 22:20:30.934  -5170   9579 -12492     828     963    -214   
3975264 2017-07-13 22:20:30.944  -5082   9531 -12473     881    1062    -189   
3975265 2017-07-13 22:20:30.964  -4968   9414 -12622    1052    1176    -154   

        activity_type  
0         Video

In [115]:
# Filter the DataFrame to keep rows where both "activity" and "duration" are not equal to 0
filtered_glasses = glasses[(glasses['activity_type'] != 0)]

# Print or further process the filtered data
print(filtered_glasses)

                  timestamp  ACC_X  ACC_Y  ACC_Z  GYRO_X  GYRO_Y  GYRO_Z  \
0   2017-06-29 08:00:00.406  -3242  10396  -6489    9117    2018    -819   
1   2017-06-29 08:00:00.426  -4014  11348 -15131    3359    -882   -2009   
2   2017-06-29 08:00:00.446  -3223  10918  -8496    -460     211   -2631   
3   2017-06-29 08:00:00.466  -5255  10690 -10854    1152     333   -3799   
4   2017-06-29 08:00:00.486   -560  14164  -8619    -327     488   -2833   
..                      ...    ...    ...    ...     ...     ...     ...   
146 2017-06-29 08:00:03.386  -2924   7065 -14436     917    -308     622   
147 2017-06-29 08:00:03.406  -2950   6926 -14427     881    -322     725   
148 2017-06-29 08:00:03.426  -2936   6949 -14567     696    -416     826   
149 2017-06-29 08:00:03.446  -2869   6967 -14336     601    -617     804   
150 2017-06-29 08:00:03.460  -3428   8258 -13647    2511    -522    1729   

    activity_type  
0     Video games  
1     In computer  
2         At home  
3     I

In [116]:
filtered_glasses

,timestamp,ACC_X,ACC_Y,ACC_Z,GYRO_X,GYRO_Y,GYRO_Z,activity_type
0,2017-06-29 08:00:00.406,-3242,10396,-6489,9117,2018,-819,Video games
1,2017-06-29 08:00:00.426,-4014,11348,-15131,3359,-882,-2009,In computer
2,2017-06-29 08:00:00.446,-3223,10918,-8496,-460,211,-2631,At home
3,2017-06-29 08:00:00.466,-5255,10690,-10854,1152,333,-3799,In computer
4,2017-06-29 08:00:00.486,-560,14164,-8619,-327,488,-2833,Eat
...,...,...,...,...,...,...,...,...
146,2017-06-29 08:00:03.386,-2924,7065,-14436,917,-308,622,Work
147,2017-06-29 08:00:03.406,-2950,6926,-14427,881,-322,725,Walk
148,2017-06-29 08:00:03.426,-2936,6949,-14567,696,-416,826,Train
149,2017-06-29 08:00:03.446,-2869,6967,-14336,601,-617,804,Walk


In [117]:
filtered_glasses['accel_x'] = filtered_glasses['ACC_X']
filtered_glasses['accel_y'] = filtered_glasses['ACC_Y']
filtered_glasses['accel_z'] = filtered_glasses['ACC_Z']
filtered_glasses['gyro_x'] = filtered_glasses['GYRO_X']
filtered_glasses['gyro_y'] = filtered_glasses['GYRO_Y']
filtered_glasses['gyro_z'] = filtered_glasses['GYRO_Z']

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_52569/3427871147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_glasses['accel_x'] = filtered_glasses['ACC_X']
/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_52569/3427871147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_glasses['accel_y'] = filtered_glasses['ACC_Y']
/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_52569/3427871147.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [118]:
filtered_glasses = filtered_glasses.drop(columns = ['ACC_Z', 'ACC_X', 'ACC_Y', 'GYRO_X', 'GYRO_Y', 'GYRO_Z'])

In [119]:
filtered_glasses

,timestamp,activity_type,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
0,2017-06-29 08:00:00.406,Video games,-3242,10396,-6489,9117,2018,-819
1,2017-06-29 08:00:00.426,In computer,-4014,11348,-15131,3359,-882,-2009
2,2017-06-29 08:00:00.446,At home,-3223,10918,-8496,-460,211,-2631
3,2017-06-29 08:00:00.466,In computer,-5255,10690,-10854,1152,333,-3799
4,2017-06-29 08:00:00.486,Eat,-560,14164,-8619,-327,488,-2833
...,...,...,...,...,...,...,...,...
146,2017-06-29 08:00:03.386,Work,-2924,7065,-14436,917,-308,622
147,2017-06-29 08:00:03.406,Walk,-2950,6926,-14427,881,-322,725
148,2017-06-29 08:00:03.426,Train,-2936,6949,-14567,696,-416,826
149,2017-06-29 08:00:03.446,Walk,-2869,6967,-14336,601,-617,804


In [120]:
filtered_data_glasses_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/filtered_data_glasses.csv'
filtered_glasses.to_csv(filtered_data_glasses_pathway, index = False)